In [1]:
#%pip install pyspedas==1.5

In [2]:
import pyspedas
import pytplot
import numpy as np
import pandas as pd

In [3]:
def print_data(x, y, x_interp, y_interp):
    if np.ndim(y) == 1:
        columns = ['y']
    else:
        columns = ['y'+str(i) for i in range(np.shape(y)[1] )]

    # Combine x and y
    df_x_y = pd.DataFrame(y, columns=columns)
    df_x_y.insert(0, 'x', x)

    for col in columns:
        df_x_y[col] = df_x_y[col].apply(lambda val: 'ORIGINAL_NAN' if pd.isna(val) else val)

    # Combine x_interp and y_interp
    df_x_y_interp = pd.DataFrame(y_interp, columns=columns)
    df_x_y_interp.insert(0, 'x_interp', x_interp)
    
    for col in columns:
        df_x_y_interp[col] = df_x_y_interp[col].apply(lambda val: 'ORIGINAL_NAN' if pd.isna(val) else val)    

    df_x_y = df_x_y.set_index('x')
    df_x_y_interp = df_x_y_interp.set_index('x_interp')

    # Merge DataFrames
    combined_df = pd.merge(df_x_y, df_x_y_interp, left_index=True, right_index=True, how='outer', suffixes=['_orig','_itrp'])
    
    # Replace NaN with blank spaces
    combined_df = combined_df.replace(np.nan, '')
    combined_df = combined_df.replace('ORIGINAL_NAN', np.nan)

    print(combined_df)

## 1. interpol
    Location: pyspedas.utilities.tinterpol
    Inner Func: scipy.interpolate.interp1d
    Input: scalar-valued array
    Usage:  pyspedas.interpol(data: List[float], data_times: List[float], out_times: List[float])
    Optional Parameters:
        - fill_value: str
            default 'extrapolate'

### 1.1 extrapolate after and before input data

In [4]:
# clean old data
%reset_selective -f "^(?!print_data$|np$|pd$|pyspedas$|pytplot$).*"

# prepare data
x1 = [4, 8, 12, 16, 20, 24]
time1 = [pytplot.time_float("2020-01-01") + i for i in x1]
x2=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
time2 = [pytplot.time_float("2020-01-01") + i for i in x2]
y1=[10, 20, 30, 40, 50, 60]

# interpolate
y_interp = pyspedas.interpol(y1, time1, time2)  #set default fill_value ('extrapolate')
print_data(x1, y1, x2, y_interp)


   y_orig  y_itrp
0             0.0
2             5.0
4    10.0    10.0
6            15.0
8    20.0    20.0
10           25.0
12   30.0    30.0
14           35.0
16   40.0    40.0
18           45.0
20   50.0    50.0
22           55.0
24   60.0    60.0
26           65.0
28           70.0


In [5]:
# clean old data
%reset_selective -f "^(?!print_data$|np$|pd$|pyspedas$|pytplot$).*"

#prepare data
x1 = [4, 8, 12, 16, 20, 24]
time1 = [pytplot.time_float("2020-01-01") + i for i in x1]
x2=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
y1=[10, 20, 30, 40, 50, 60]
time2 = [pytplot.time_float("2020-01-01") + i for i in x2]

# interpolate
y_interp = pyspedas.interpol(y1, time1, time2, fill_value='nan')  #set fill_value, add nan when extrapolating
print_data(x1, y1, x2, y_interp)

   y_orig  y_itrp
0             NaN
2             NaN
4    10.0    10.0
6            15.0
8    20.0    20.0
10           25.0
12   30.0    30.0
14           35.0
16   40.0    40.0
18           45.0
20   50.0    50.0
22           55.0
24   60.0    60.0
26            NaN
28            NaN


### 1.2 interpolate data with nan in the middle

In [6]:
# clean old data
%reset_selective -f "^(?!print_data$|np$|pd$|pyspedas$|pytplot$).*"

# prepare data
x1 = [4, 8, 12, 16, 20, 24]
time1 = [pytplot.time_float("2020-01-01") + i for i in x1]
x2=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
time2 = [pytplot.time_float("2020-01-01") + i for i in x2]
y2=[10, 20, np.nan, np.nan, 50, 60]

# interpolate
y_interp = pyspedas.interpol(y2, time1, time2)
print_data(x1, y2, x2, y_interp)

   y_orig  y_itrp
0             0.0
2             5.0
4    10.0    10.0
6            15.0
8    20.0    20.0
10            NaN
12    NaN     NaN
14            NaN
16    NaN     NaN
18            NaN
20   50.0     NaN
22           55.0
24   60.0    60.0
26           65.0
28           70.0


❗note that 50 is not showing in the interpolation result

In [7]:
# clean old data
%reset_selective -f "^(?!print_data$|np$|pd$|pyspedas$|pytplot$).*"

# prepare data
x1 = [4, 8, 12, 16, 20, 24]
time1 = [pytplot.time_float("2020-01-01") + i for i in x1]
x2=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
time2 = [pytplot.time_float("2020-01-01") + i for i in x2]
y2=[10, 20, np.nan, np.nan, 50, 60]

# interpolate
y_interp = pyspedas.interpol(y2, time1, time2, fill_value='nan')
print_data(x1, y2, x2, y_interp)

   y_orig  y_itrp
0             NaN
2             NaN
4    10.0    10.0
6            15.0
8    20.0    20.0
10            NaN
12    NaN     NaN
14            NaN
16    NaN     NaN
18            NaN
20   50.0    50.0
22           55.0
24   60.0    60.0
26            NaN
28            NaN


## 2. tinterpol
    Location: pyspedas.analysis.tinterpol
    Input: tplot variables
    Inner Func: xarray.DataArray.interp
    Usage: pyspedas.tinterpol(names: str | List[str], interp_to: str | List[str])
    Optional Parameters: 
        - method: str
            Default is Linear. Options: 'linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'previous', 'next'
        - newname: str | List[str]
            Default is ''
        - suffix: str
            Default is '-itrp'

create tplot variables

### 2.1 extrapolate data before and after input date

In [8]:
# clean old data
%reset_selective -f "^(?!print_data$|np$|pd$|pyspedas$|pytplot$).*"

# prepare date
x1 = [4, 8, 12, 16, 20, 24]
time1 = [pytplot.time_float("2020-01-01") + i for i in x1]
x2=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
time2 = [pytplot.time_float("2020-01-01") + i for i in x2]
y1=[10, 20, 30, 40, 50, 60]
pytplot.store_data('x1', data={'x': time1, 'y': y1})
pytplot.store_data('x2', data={'x': time2, 'y': np.zeros_like(time2)})  # y will be ignored

# interpolate
pyspedas.tinterpol(names='x1', interp_to = 'x2', newname='x1_interp')
x1_interp = pytplot.get_data('x1_interp')
print_data(x1, y1, x2, x1_interp.y)

20-Jul-24 12:18:08: tinterpol (linear) was applied to: x1_interp


   y_orig  y_itrp
0             NaN
2             NaN
4    10.0    10.0
6            15.0
8    20.0    20.0
10           25.0
12   30.0    30.0
14           35.0
16   40.0    40.0
18           45.0
20   50.0    50.0
22           55.0
24   60.0    60.0
26            NaN
28            NaN


### 2.2 interpolate data with nan in the middle

In [9]:
# clean old data
%reset_selective -f "^(?!print_data$|np$|pd$|pyspedas$|pytplot$).*"

# prepare data
x1 = [4, 8, 12, 16, 20, 24]
time1 = [pytplot.time_float("2020-01-01") + i for i in x1]
x2=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
time2 = [pytplot.time_float("2020-01-01") + i for i in x2]
y2=[10, 20, np.nan, np.nan, 50, 60]
pytplot.store_data('x1_nan', data={'x': time1, 'y': y2})
pytplot.store_data('x2', data={'x': time2, 'y': np.zeros_like(time2)})  # y will be ignored

# interpolate
pyspedas.tinterpol(names='x1_nan', interp_to = 'x2', newname='x1_nan_interp')
x1_interp = pytplot.get_data('x1_nan_interp')
print_data(x1, y2, x2, x1_interp.y)

20-Jul-24 12:18:40: tinterpol (linear) was applied to: x1_nan_interp


   y_orig  y_itrp
0             NaN
2             NaN
4    10.0    10.0
6            15.0
8    20.0    20.0
10            NaN
12    NaN     NaN
14            NaN
16    NaN     NaN
18            NaN
20   50.0    50.0
22           55.0
24   60.0    60.0
26            NaN
28            NaN


### 2.3 higher dimension of input data

In [10]:
# clean old data
%reset_selective -f "^(?!print_data$|np$|pd$|pyspedas$|pytplot$).*"

# prepare data
x1 = [4, 8, 12, 16, 20, 24]
time1 = [pytplot.time_float("2020-01-01") + i for i in x1]
x2=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
time2 = [pytplot.time_float("2020-01-01") + i for i in x2]
y3 = [[np.nan,1,1],[np.nan,2,3],[4,np.nan,47],[4,np.nan,5],[5,5,99],[7,np.nan,np.nan]]

# interpolate
pytplot.store_data('x1_multi', data={'x':time1, 'y': y3})
pyspedas.tinterpol(names='x1_multi', interp_to = 'x2', newname='x1_mult_interp')
x1_interp = pytplot.get_data('x1_mult_interp')
print_data(x1, y3, x2, x1_interp.y)


20-Jul-24 12:18:45: tinterpol (linear) was applied to: x1_mult_interp


   y0_orig y1_orig y2_orig  y0_itrp  y1_itrp  y2_itrp
0                               NaN      NaN      NaN
2                               NaN      NaN      NaN
4      NaN     1.0     1.0      NaN      1.0      1.0
6                               NaN      1.5      2.0
8      NaN     2.0     3.0      NaN      2.0      3.0
10                              NaN      NaN     25.0
12     4.0     NaN    47.0      NaN      NaN     47.0
14                              4.0      NaN     26.0
16     4.0     NaN     5.0      4.0      NaN      5.0
18                              4.5      NaN     52.0
20     5.0     5.0    99.0      5.0      NaN     99.0
22                              6.0      NaN      NaN
24     7.0     NaN     NaN      7.0      NaN      NaN
26                              NaN      NaN      NaN
28                              NaN      NaN      NaN


❗note that x=12, y=4.0 should be shown in y0_itrp  
❗note that x=20, y=5.0 should be shown in y1_itrp

### 2.4 multiple inputs

In [19]:
pyspedas.tinterpol(names=['x1','x1_multi'], interp_to = 'x2', newname=['x1_interp','x1_multi_interp'])

25-Jun-24 11:21:45: tinterpol (linear) was applied to: x1_interp
25-Jun-24 11:21:45: tinterpol (linear) was applied to: x1_multi_interp


## 3. interp_nan
    Location: pytplot.tplot_math.interp_nan
    Input: tplot variables
    Inner Func: xarray.DataArray.interpolate_na
    Usage: pytplot.interp_nan(tvar: str)
    Optional Parameters: 
        - s_limit: int | float, the maximum size of the gap in seconds to not interpolate over
        - newname: str, name of new tvar for added data. If not set, then the original tvar is replaced.


### 3.1 interp nan for 1d data

In [20]:
# clean old data
%reset_selective -f "^(?!print_data$|np$|pd$|pyspedas$|pytplot$).*"

# prepare data
x1 = [4, 8, 12, 16, 20, 24, 28]
time1 = [pytplot.time_float("2020-01-01") + i for i in x1]
y2=[np.nan, 20, np.nan, np.nan, np.nan, 60, np.nan]
pytplot.store_data('x1_nan', data={'x': time1, 'y': y2})

# interpolate
pytplot.interp_nan('x1_nan', newname='x1_naninterp', s_limit=2)
x1_interp = pytplot.get_data('x1_naninterp')
print_data(x1, y2, x1, x1_interp.y)

    y_orig  y_itrp
x                 
4      NaN     NaN
8     20.0    20.0
12     NaN    30.0
16     NaN    40.0
20     NaN     NaN
24    60.0    60.0
28     NaN     NaN


❗note only two Nan is interpolated because s_limit=2

### 3.2 interp nan for multi-dimen data

In [21]:
# clean old data
%reset_selective -f "^(?!print_data$|np$|pd$|pyspedas$|pytplot$).*"

# prepare data
x1 = [4, 8, 12, 16, 20, 24]
time1 = [pytplot.time_float("2020-01-01") + i for i in x1]
y3 = [[np.nan,1,1],[np.nan,2,3],[4,np.nan,np.nan],[4,np.nan,np.nan],[5,np.nan,99],[7,6,np.nan]]
pytplot.store_data('x1_multi', data={'x':time1, 'y': y3})

# interpolate
pyspedas.interp_nan('x1_multi', newname='x1_multinterp', s_limit=2)
x1_interp = pytplot.get_data('x1_multinterp')
print_data(x1, y3, x1, x1_interp.y)

    y0_orig  y1_orig  y2_orig  y0_itrp  y1_itrp  y2_itrp
x                                                       
4       NaN      1.0      1.0      NaN      1.0      1.0
8       NaN      2.0      3.0      NaN      2.0      3.0
12      4.0      NaN      NaN      4.0      3.0     35.0
16      4.0      NaN      NaN      4.0      4.0     67.0
20      5.0      NaN     99.0      5.0      NaN     99.0
24      7.0      6.0      NaN      7.0      6.0      NaN


## 4. tinterp
    Location: pytplot.tplot_math.tinterp 
    Input: tplot variables  
    Inner Func: xarray.DataArray.interp_like   
    Usage: pytplot.tinterp(tvar1: str, tvar2: str) # interp tvar1 according to the time of tvar2
    Optional Parameters: 
        - replace: bool, If true, the data in the original tplot variable is replaced.  Otherwise, a variable is created.

In [22]:
pytplot.store_data('a', data={'x':[0,4,8,12,16], 'y':[1,2,3,4,5]})
pytplot.store_data('c', data={'x':[0,4,8,12,16,19,21], 'y':[1,4,1,7,1,9,1]})
pytplot.tinterp('a','c')
print(pytplot.data_quants['a_tinterp'].data)

[1. 4. 1. 7. 1.]


❗note that the example in the docstring in tinterp.py needs modification  
>>> pytplot.store_data('a', data={'x':[0,4,8,12,16], 'y':[1,2,3,4,5]})  
>>> pytplot.store_data('c', data={'x':[0,4,8,12,16,19,21], 'y':[1,4,1,7,1,9,1]})  
>>> pytplot.tinterp('a','c')  
>>> print(pytplot.data_quants['c_interp'].data)  

### 4.1 extrapolate data before and after input date

In [23]:
# clean old data
%reset_selective -f "^(?!print_data$|np$|pd$|pyspedas$|pytplot$).*"

# prepare data
x1 = [4, 8, 12, 16, 20, 24]
time1 = [pytplot.time_float("2020-01-01") + i for i in x1]
x2=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
time2 = [pytplot.time_float("2020-01-01") + i for i in x2]
y1=[10, 20, 30, 40, 50, 60]
pytplot.store_data('x1', data={'x': time1, 'y': y1})
pytplot.store_data('x2', data={'x': time2, 'y': np.zeros_like(time2)})  # y will be ignored

# interpolate
pyspedas.tinterpol('x1', 'x2')
x1_interp = pytplot.get_data('x1_interp')
print_data(x1, y1, x2, x1_interp.y)

25-Jun-24 11:21:46: tinterpol (linear) was applied to: x1-itrp


   y_orig  y_itrp
0             NaN
2             NaN
4    10.0    10.0
6            15.0
8    20.0    20.0
10           25.0
12   30.0    30.0
14           35.0
16   40.0    40.0
18           45.0
20   50.0    50.0
22           55.0
24   60.0    60.0
26            NaN
28            NaN


### 4.2 interpolate data with nan in the middle

In [24]:
# clean old data
%reset_selective -f "^(?!print_data$|np$|pd$|pyspedas$|pytplot$).*"

# prepare data
x1 = [4, 8, 12, 16, 20, 24]
time1 = [pytplot.time_float("2020-01-01") + i for i in x1]
x2=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
time2 = [pytplot.time_float("2020-01-01") + i for i in x2]
y2=[10, 20, np.nan, np.nan, 50, 60]
pytplot.store_data('x1_nan', data={'x': time1, 'y': y2})
pytplot.store_data('x2', data={'x': time2, 'y': np.zeros_like(time2)})  # y will be ignored

# interpolate
pyspedas.tinterpol('x1_nan', 'x2')
x1_interp = pytplot.get_data('x1_nan_interp')
print_data(x1, y2, x2, x1_interp.y)

25-Jun-24 11:21:46: tinterpol (linear) was applied to: x1_nan-itrp


   y_orig  y_itrp
0             NaN
2             NaN
4    10.0    10.0
6            15.0
8    20.0    20.0
10            NaN
12    NaN     NaN
14            NaN
16    NaN     NaN
18            NaN
20   50.0    50.0
22           55.0
24   60.0    60.0
26            NaN
28            NaN
